In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import re
import json
import math
import statistics
import matplotlib.pyplot as plt
%matplotlib inline

import bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearColorMapper
from bokeh.models.tools import HoverTool
output_notebook()

Loading BokehJS ...

## Electoral College Advantage Over Time

Scrape historical election results from the [US Elections Atlas](https://uselectionatlas.org/) to algorithmically find each election year's tipping point state, and the relative advantage toward Democrats or Republicans in that year's Electoral College. 

---

#### Year-party candidates crosswalk

Manually collected/cleaned from Wikipedia.

In [2]:
pres_candidates = pd.read_csv('data/presidential_candidates.csv')
pres_candidates['dem_cand'] = pres_candidates.dem_candidate.str.lower().str.strip('†‡ii ').apply(lambda x: 
                                                                                                   x.split(' ')[-1])
pres_candidates['rep_cand'] = pres_candidates.rep_candidate.str.lower().str.strip('†‡ii ').apply(lambda x: 
                                                                                                   x.split(' ')[-1])
pres_candidates.loc[pres_candidates['dem_cand'].str.len() > 7, 'dem_cand'] = pres_candidates.loc[pres_candidates['dem_cand'].str.len() > 7, 'dem_cand'].apply(lambda x: x[:6] + '.')
pres_candidates.loc[pres_candidates['rep_cand'].str.len() > 7, 'rep_cand'] = pres_candidates.loc[pres_candidates['rep_cand'].str.len() > 7, 'rep_cand'].apply(lambda x: x[:6] + '.')
pres_candidates = pres_candidates[['year', 'dem_cand', 'rep_cand']].melt(id_vars=['year'], var_name='party', value_name='candidate')
pres_candidates.party = pres_candidates.party.apply(lambda x: x.split('_')[0])
pres_candidates

,year,party,candidate
0,1900,dem,bryan
1,1904,dem,parker
2,1908,dem,bryan
3,1912,dem,wilson
4,1916,dem,wilson
...,...,...,...
57,2004,rep,bush
58,2008,rep,mccain
59,2012,rep,romney
60,2016,rep,trump


In [3]:
def get_max_ev_candidate(row, ev_cols):
    """
    From separate EV total columns, find the party that won the most
    EVs in that state.
    """
    return ev_cols[row.argmax()].split('_')[-1]

In [4]:
def get_popular_vote_margin(df):
    """
    Scaled to be relative to Dems (+ means Dems win, - means Reps win).
    """
    dem_voteshare = df.dem_votes.sum() / df.totalvote.sum()
    rep_voteshare = df.rep_votes.sum() / df.totalvote.sum()
    if dem_voteshare > rep_voteshare:
        print('dem wins popular vote')
    elif rep_voteshare > dem_voteshare:
        print('rep wins popular vote')
    else:
        print('popular vote tied')
    return dem_voteshare - rep_voteshare


In [5]:
def get_tipping_point_state(df):
    """
    Calculate tipping point state, after sorting vote margin by 
    winning party and calculating the critical number of EVs.
    """
    tp_state = df.copy()
    tp_state['dem_margin'] = tp_state.dem_voteshare - tp_state.rep_voteshare
    tp_state['rep_margin'] = tp_state.rep_voteshare - tp_state.dem_voteshare
    tp_state = tp_state.sort_values(f'{get_election_winner(df)}_margin', ascending=False)
    tp_state['cum_ev'] = tp_state.ev_total.cumsum()
    winning_ev = math.ceil(tp_state.ev_total.sum()/2)
    tipping_state = tp_state[tp_state.cum_ev > winning_ev]['state'].iloc[0]
    tipping_state_margin = tp_state[tp_state.state == tipping_state][f'dem_margin'].iloc[0]
    
    print('tp state:', tipping_state)
    print('tp state margin', tipping_state_margin)
    return tipping_state, tipping_state_margin

In [6]:
def get_election_winner(df):
    """
    Find the party that won the most overall EVs.
    """
    return df.groupby('winner')['ev_total'].sum().idxmax()

In [7]:
def scrape_atlas_year(year):
    """
    Scrape state-level election results from the US Elections atlas, by year.
    Returns raw soup of election returns table.
    """
    url = f'https://uselectionatlas.org/RESULTS/data.php?year={year}&datatype=national&def=1'
    print(url)
    r = requests.get(url)
    print(r.status_code)
    soup = BeautifulSoup(r.text, 'html.parser')
    data_raw = soup.find_all('table')[-1].find_all('tr')
    print(len(data_raw))
    return data_raw

In [8]:
def generate_atlas_table(data_raw, pres_candidates_df):
    """
    From raw soup of election returns, generate a clean dataframe
    containing EV counts, voteshares and raw vote totals by state.
    """
    raw_text = []
    raw_color = []

    for d in data_raw[0].find_all('td'):
        raw_text.append(d.text.lower().replace('\xa0', ''))
        if len(d.find_all('span')) > 0:
            raw_color.append(d.find_all('span')[0]['style'].strip('color:'))
        else:
            raw_color.append('')
            
    cands = raw_text[raw_text.index('%margin')+1 : raw_text.index('other')]
    
    # set dynamically based on year
    candidate_values = pres_candidates[pres_candidates.year == year]['candidate'].values
    party_values = pres_candidates[pres_candidates.year == year]['party'].values
    cand_party_subs = dict(zip(candidate_values, party_values))
    
    # cand color mapping
    cands = raw_text[raw_text.index('%margin')+1 : raw_text.index('other')]
    cands = [cand_party_subs[c] if c in cand_party_subs.keys() else c for c in cands]
    colors = raw_color[raw_text.index('%margin')+1 : raw_text.index('other')]
    cands_color_mapping = dict(zip(colors, cands))
    cands_color_mapping['#000000'] = 'other'
    
    # loop over raw text columns, to generate clean col names
    for i, text in enumerate(raw_text):
        if len(raw_color[i]) > 0 and text == 'ev' and raw_color[i] in ['#DD0806', '#0000D4']:
            raw_text[i] = text + '_' + cands_color_mapping[raw_color[i]]
        if text in cand_party_subs.keys():
            raw_text[i] = cand_party_subs[text]
    counts = {}
    col_names_clean = []

    for n in raw_text:
        if n in counts:
            counts[n] += 1
            col_names_clean.append(n + '_votes')
        else:
            counts[n] = 1
            col_names_clean.append(n)
            
    # set df w/ clean col names
    rows_raw = []
    for i in np.arange(0, len(data_raw)):
        new_row = [t.text.replace('\xa0', '') for t in data_raw[i].find_all('td')]
        rows_raw.append(new_row)
    rows_raw.pop(0)  
    df = pd.DataFrame(rows_raw, columns=col_names_clean)
    df = df[df.state.isin([None, 'Total', 'Enable']) == False]
    df = df.reset_index(drop=True)    
    df['ev_total'] = df[df.columns[df.columns.str.startswith('ev')]].astype(int).sum(axis=1)
    df = df.rename(columns={'dem': 'dem_voteshare', 'rep': 'rep_voteshare'})
    df = df[['state'] + 
            df.columns[df.columns.str.startswith('ev')].tolist() + 
            ['totalvote', 'dem_voteshare', 'rep_voteshare', 'dem_votes', 'rep_votes']]

    # find state winners
    for numeric_col in [c for c in df.columns if c not in ['state', 'ev_total']]:
        df[numeric_col] = df[numeric_col].str.replace(',', '').str.replace('%', '').astype(float)   
    ev_cols = [c for c in df.columns[df.columns.str.startswith('ev')] if c != 'ev_total']
    df['winner'] = df[ev_cols].apply(lambda row: get_max_ev_candidate(row, ev_cols), axis=1)
    
    print('df shape:', df.shape)
    print('dem EVs:', df.ev_dem.sum())
    print('rep EVs:', df.ev_rep.sum())
    print('total vote:', df.totalvote.sum())
    return df

---

#### Loop over years

In [9]:
years = list(np.arange(1900, 2024, 4))
print(len(years))
years[:3]

31


[1900, 1904, 1908]

In [10]:
df_final = pd.DataFrame(columns=['year', 'tp_state', 'tp_state_margin',
                                 'natl_margin', 'ec_advantage', 'winning_party'])
for year in years:
    print(year)
    data_raw = scrape_atlas_year(year)
    df = generate_atlas_table(data_raw, pres_candidates)
    winning_party = get_election_winner(df)
    natl_margin = get_popular_vote_margin(df)
    tp_state, tp_state_margin = get_tipping_point_state(df)
    new_df = pd.DataFrame([[year, tp_state, tp_state_margin/100, natl_margin,  
                            tp_state_margin/100 - natl_margin, winning_party]]
                            , columns=['year', 'tp_state', 'tp_state_margin', 
                                       'natl_margin', 'ec_advantage', 'winning_party'])
    df_final = pd.concat([df_final, new_df], sort=False)

1900
https://uselectionatlas.org/RESULTS/data.php?year=1900&datatype=national&def=1
200
49
df shape: (45, 10)
dem EVs: 155.0
rep EVs: 292.0
total vote: 13971846.0
rep wins popular vote
tp state: Illinois
tp state margin -8.39
1904
https://uselectionatlas.org/RESULTS/data.php?year=1904&datatype=national&def=1
200
49
df shape: (45, 10)
dem EVs: 140.0
rep EVs: 336.0
total vote: 13525759.0
rep wins popular vote
tp state: New Jersey
tp state margin -18.630000000000003
1908
https://uselectionatlas.org/RESULTS/data.php?year=1908&datatype=national&def=1
200
50
df shape: (46, 10)
dem EVs: 162.0
rep EVs: 321.0
total vote: 14889109.0
rep wins popular vote
tp state: West Virginia
tp state margin -10.25
1912
https://uselectionatlas.org/RESULTS/data.php?year=1912&datatype=national&def=1
200
52
df shape: (48, 11)
dem EVs: 435.0
rep EVs: 8.0
total vote: 15046540.0
dem wins popular vote
tp state: Ohio
tp state margin 14.14
1916
https://uselectionatlas.org/RESULTS/data.php?year=1916&datatype=national&de

In [11]:
df_final = df_final.reset_index(drop=True)
df_final = pd.merge(df_final, 
                    pres_candidates.rename(columns={'party': 'winning_party'}), 
                    how='left', 
                    on=['year', 'winning_party'])
df_final = df_final.round(4)
df_final['ec_advantage_display'] = df_final.ec_advantage.apply(lambda x: str(round((x*100), 1)) + '%')
df_final['winning_party_display'] = df_final.winning_party.str.title()
df_final['candidate_display'] = df_final.candidate.str.title()

print(df_final.shape)
df_final

(31, 10)


,year,tp_state,tp_state_margin,natl_margin,ec_advantage,winning_party,candidate,ec_advantage_display,winning_party_display,candidate_display
0,1900,Illinois,-0.0839,-0.0616,-0.0223,rep,mckinl.,-2.2%,Rep,Mckinl.
1,1904,New Jersey,-0.1863,-0.1882,0.0019,rep,roosev.,0.2%,Rep,Roosev.
2,1908,West Virginia,-0.1025,-0.0852,-0.0173,rep,taft,-1.7%,Rep,Taft
3,1912,Ohio,0.1414,0.1865,-0.0451,dem,wilson,-4.5%,Dem,Wilson
4,1916,California,0.0038,0.0313,-0.0275,dem,wilson,-2.8%,Dem,Wilson
5,1920,Rhode Island,-0.3119,-0.2623,-0.0496,rep,harding,-5.0%,Rep,Harding
6,1924,New York,-0.2663,-0.2521,-0.0142,rep,coolid.,-1.4%,Rep,Coolid.
7,1928,Illinois,-0.1465,-0.1742,0.0277,rep,hoover,2.8%,Rep,Hoover
8,1932,Iowa,0.1771,0.1776,-0.0005,dem,roosev.,-0.1%,Dem,Roosev.
9,1936,Ohio,0.2055,0.2426,-0.0371,dem,roosev.,-3.7%,Dem,Roosev.


In [12]:
# df_final.to_csv('data/tipping_point_states.csv', index=False)

In [13]:
# double check tp states for years:
# 1912
# 1924
# 1948 
# 1960

---

### Interactive plot

In [14]:
df_final = pd.read_csv('data/tipping_point_states.csv')

In [15]:
color_mapper = LinearColorMapper(palette=tuple(reversed(bokeh.palettes.RdBu11)), 
                                 low=-0.05,
                                 high=0.05)
plot1 = figure(title='Electoral College Advantage Over Time',
             x_axis_label='Year',
             y_axis_label='Dem. tipping point state margin - national Dem. margin',
             width=800,
             height=500)
r = plot1.line('year', 'ec_advantage',
           source=df_final,
           color='grey', 
           alpha=1, 
           line_width=2,
           line_join='bevel')
plot1.scatter('year', 'ec_advantage', 
          source=df_final,
          line_color="black", 
          color={'field': 'ec_advantage', 'transform': color_mapper},  
          fill_alpha=0.8, 
          size=14)
plot1.add_tools(HoverTool(
                tooltips=[
                    ('Election Year', '@year'),
                    ('EC Advantage', '@ec_advantage_display'),
                    ('Winning Party', '@winning_party_display'),
                    ('President', '@candidate_display')
                ]
            ))
show(plot1)

---

## Electoral Structural Biases Over Time

Using data from the [MIT Election Data + Science Lab](https://electionlab.mit.edu/data), explore how biases in the Electoral College compared with the U.S. House and Senate vary across election years.

In [16]:
def get_election_natl_margin(year):
    """
    Scaled to be relative to Dems (+ means Dems win, - means Reps win).
    """
    return tp_states[tp_states.year == round_election_year(year)]['natl_margin'].iloc[0]

In [17]:
def round_election_year(year):
    """
    For a given election year, return the nearest previous year 
    with a presidential election.
    """
    return year - (year % 4)

In [18]:
def code_winning_party_column(row):
    if row['dem_votes'] > row['rep_votes']:
        return 'dem'
    elif row['dem_votes'] < row['rep_votes']:
        return 'rep'
    else:
        return None

In [19]:
tp_states = pd.read_csv('data/tipping_point_states.csv')
tp_states.head()

,year,tp_state,tp_state_margin,natl_margin,ec_advantage,winning_party,candidate,ec_advantage_display,winning_party_display,candidate_display
0,1900,Illinois,-0.0839,-0.0616,-0.0223,rep,mckinl.,-2.2%,Rep,Mckinl.
1,1904,New Jersey,-0.1863,-0.1882,0.0019,rep,roosev.,0.2%,Rep,Roosev.
2,1908,West Virginia,-0.1025,-0.0852,-0.0173,rep,taft,-1.7%,Rep,Taft
3,1912,Ohio,0.1414,0.1865,-0.0451,dem,wilson,-4.5%,Dem,Wilson
4,1916,California,0.0038,0.0313,-0.0275,dem,wilson,-2.8%,Dem,Wilson


### House results

In [20]:
house = pd.read_csv('data/1976-2018-house.csv', encoding='latin-1')
print(house.shape)
house.head()

(29636, 20)


,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,runoff,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version,fusion_ticket
0,1976,ALABAMA,AL,1,63,41,US HOUSE,1,gen,False,False,BILL DAVENPORT,DEMOCRAT,False,TOTAL,58906,157170,False,20201202,False
1,1976,ALABAMA,AL,1,63,41,US HOUSE,1,gen,False,False,JACK EDWARDS,REPUBLICAN,False,TOTAL,98257,157170,False,20201202,False
2,1976,ALABAMA,AL,1,63,41,US HOUSE,1,gen,False,False,NaN,NaN,True,TOTAL,7,157170,False,20201202,False
3,1976,ALABAMA,AL,1,63,41,US HOUSE,2,gen,False,False,J. CAROLE KEAHEY,DEMOCRAT,False,TOTAL,66288,156362,False,20201202,False
4,1976,ALABAMA,AL,1,63,41,US HOUSE,2,gen,False,False,NaN,NaN,True,TOTAL,5,156362,False,20201202,False


In [21]:
def get_house_bias(year, house_df):
    """
    For a given year, return a row with 
    the Dem. twoway margin of the median house district.
    """
    df = house_df[house_df.year == year]
    df = df[df.party.isin(['DEMOCRAT', 'REPUBLICAN'])]
    df['cand_rank'] = df.groupby(['state_po', 'district'])['candidatevotes'].rank(ascending=False)
    df = df[df.cand_rank <= 2]
    df['cand_party_rank'] = df.groupby(['state_po', 'district', 'party'])['candidatevotes'].rank(ascending=False)
    df = df[df.cand_party_rank == 1]
    df = df.pivot(index=['year', 'state_po', 'district', 'totalvotes'], columns=['party'], values=['candidatevotes']).reset_index()
    df.columns = ['year', 'state_code', 'district_code', 'totalvotes', 'dem_votes', 'rep_votes']
    df = df.fillna(0)
    df['cd'] = df.state_code + '-' + df.district_code.astype(str).str.zfill(2)
    df['dem_twoway'] = df.dem_votes / (df.dem_votes + df.rep_votes)
    df['dem_voteshare'] = df.dem_votes / df.totalvotes
    df['rep_voteshare'] = df.rep_votes / df.totalvotes
    df['dem_margin'] = df.dem_voteshare - df.rep_voteshare
    df['rep_margin'] = df.rep_voteshare - df.dem_voteshare
    df['winning_party'] = df.apply(lambda x: code_winning_party_column(x), axis=1)
    
    house_median_dem_margin = df.sort_values('dem_margin', ascending=False).iloc[217]['dem_margin']
    house_median_cd = df.sort_values('dem_margin', ascending=False).iloc[217]['cd']
    house_median_winning_party = df.sort_values('dem_margin', ascending=False).iloc[217]['winning_party']
    house_median_state = df.sort_values('dem_margin', ascending=False).iloc[217]['state_code']
    new_row = [year, house_median_state, house_median_cd, house_median_dem_margin, house_median_winning_party]
    return new_row

In [22]:
rows_raw = []
for year in np.arange(1976, 2020, 2):
    print(year)
    new_row = get_house_bias(year, house)
    rows_raw.append(new_row)

1976
1978
1980
1982
1984
1986
1988
1990
1992
1994
1996
1998
2000
2002
2004
2006
2008
2010
2012
2014
2016
2018


In [23]:
house_bias = pd.DataFrame(rows_raw, columns=['year', 'house_median_state', 'house_median_cd', 
                                             'house_median_dem_margin', 'house_median_winning_party'])
house_bias['natl_margin'] = house_bias.year.apply(get_election_natl_margin)
house_bias['house_advantage'] = house_bias.house_median_dem_margin - house_bias.natl_margin
house_bias

,year,house_median_state,house_median_cd,house_median_dem_margin,house_median_winning_party,natl_margin,house_advantage
0,1976,MI,MI-08,0.186485,dem,0.0206,0.165885
1,1978,WI,WI-02,0.163340,dem,0.0206,0.142740
2,1980,IN,IN-02,0.081177,dem,-0.0974,0.178577
3,1982,MA,MA-06,0.156564,dem,-0.0974,0.253964
4,1984,WV,WV-02,0.119416,dem,-0.1822,0.301616
5,1986,CA,CA-34,0.206654,dem,-0.1822,0.388854
6,1988,WV,WV-04,0.226026,dem,-0.0773,0.303326
7,1990,NC,NC-04,0.161358,dem,-0.0773,0.238658
8,1992,MI,MI-09,0.088799,dem,0.0556,0.033199
9,1994,ME,ME-01,-0.037839,rep,0.0556,-0.093439


In [24]:
# house.groupby(['year'])['district'].nunique()

---

### Senate results

In [25]:
senate = pd.read_csv('data/1976-2020-senate.csv', encoding='latin-1')
print(senate.shape)
senate.head()

(3629, 19)


,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party_detailed,writein,mode,candidatevotes,totalvotes,unofficial,version,party_simplified
0,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,SAM STEIGER,REPUBLICAN,False,total,321236,741210,False,20210114,REPUBLICAN
1,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,WM. MATHEWS FEIGHAN,INDEPENDENT,False,total,1565,741210,False,20210114,OTHER
2,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,DENNIS DECONCINI,DEMOCRAT,False,total,400334,741210,False,20210114,DEMOCRAT
3,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,ALLAN NORWITZ,LIBERTARIAN,False,total,7310,741210,False,20210114,LIBERTARIAN
4,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,BOB FIELD,INDEPENDENT,False,total,10765,741210,False,20210114,OTHER


In [26]:
# fix bugs
senate.loc[(senate.year == 2020) & (senate.candidate == 'CYNTHIA M. LUMMIS'), 'party_simplified'] = 'REPUBLICAN'
senate.loc[(senate.year == 2020) & (senate.candidate == 'MERAV BEN DAVID'), 'party_simplified'] = 'DEMOCRAT'

In [27]:
# senate.groupby(['year', 'special'])['state_po'].nunique()

In [28]:
def get_senate_bias(year, senate_df):
    """
    For a given year, return a row with 
    the Dem. twoway margin of the median senate race.
    """
    df = senate[senate.year == year]
    df = df[df.party_simplified.isin(['DEMOCRAT', 'REPUBLICAN'])]
    df['cand_rank'] = df.groupby(['state_po', 'special'])['candidatevotes'].rank(ascending=False)
    df = df[df.cand_rank <= 2]
    df['cand_party_rank'] = df.groupby(['state_po', 'special', 'party_simplified'])['candidatevotes'].rank(ascending=False)
    df = df[df.cand_party_rank == 1]
    df = df.pivot(index=['year', 'state_po', 'special', 'totalvotes'], columns=['party_simplified'], values=['candidatevotes']).reset_index()
    df.columns = ['year', 'state_code', 'special', 'totalvotes', 'dem_votes', 'rep_votes']
    df = df.fillna(0)
    df['dem_twoway'] = df.dem_votes / (df.dem_votes + df.rep_votes)
    df['dem_voteshare'] = df.dem_votes / df.totalvotes
    df['rep_voteshare'] = df.rep_votes / df.totalvotes
    df['dem_margin'] = df.dem_voteshare - df.rep_voteshare
    df['winning_party'] = df.apply(lambda x: code_winning_party_column(x), axis=1)
    
    median_idx = math.floor(df.shape[0]/2)
    senate_median_dem_margin = df.sort_values('dem_margin', ascending=False).iloc[median_idx]['dem_margin']
    senate_median_winning_party = df.sort_values('dem_margin', ascending=False).iloc[median_idx]['winning_party']
    senate_median_state = df.sort_values('dem_margin', ascending=False).iloc[median_idx]['state_code']
    new_row = [year, senate_median_state, senate_median_dem_margin, senate_median_winning_party]
    return new_row

In [29]:
rows_raw = []
for year in np.arange(1976, 2022, 2):
    print(year)
    new_row = get_senate_bias(year, senate)
    rows_raw.append(new_row)

1976
1978
1980
1982
1984
1986
1988
1990
1992
1994
1996
1998
2000
2002
2004
2006
2008
2010
2012
2014
2016
2018
2020


In [30]:
senate_bias = pd.DataFrame(rows_raw, columns=['year', 'senate_median_state', 
                                             'senate_median_dem_margin', 'senate_median_winning_party'])
senate_bias['natl_margin'] = senate_bias.year.apply(get_election_natl_margin)
senate_bias['senate_advantage'] = senate_bias.senate_median_dem_margin - senate_bias.natl_margin
senate_bias

,year,senate_median_state,senate_median_dem_margin,senate_median_winning_party,natl_margin,senate_advantage
0,1976,NY,0.107002,dem,0.0206,0.086402
1,1978,TX,-0.005287,rep,0.0206,-0.025887
2,1980,GA,-0.017434,rep,-0.0974,0.079966
3,1982,OH,0.155200,dem,-0.0974,0.252600
4,1984,IL,0.018617,dem,-0.1822,0.200817
5,1986,CA,0.014174,dem,-0.1822,0.196374
6,1988,WI,0.045284,dem,-0.0773,0.122584
7,1990,MN,0.026340,dem,-0.0773,0.103640
8,1992,CA,0.049009,dem,0.0556,-0.006591
9,1994,VT,-0.097495,rep,0.0556,-0.153095


In [31]:
advantage_df = pd.merge(senate_bias, house_bias.drop(['natl_margin'], axis=1), how='left', on='year')
advantage_df = pd.merge(advantage_df, tp_states[['year', 'ec_advantage']], how='left', on='year')
print(advantage_df.shape)
advantage_df.head()

(23, 12)


,year,senate_median_state,senate_median_dem_margin,senate_median_winning_party,natl_margin,senate_advantage,house_median_state,house_median_cd,house_median_dem_margin,house_median_winning_party,house_advantage,ec_advantage
0,1976,NY,0.107002,dem,0.0206,0.086402,MI,MI-08,0.186485,dem,0.165885,-0.0039
1,1978,TX,-0.005287,rep,0.0206,-0.025887,WI,WI-02,0.163340,dem,0.142740,NaN
2,1980,GA,-0.017434,rep,-0.0974,0.079966,IN,IN-02,0.081177,dem,0.178577,0.0181
3,1982,OH,0.155200,dem,-0.0974,0.252600,MA,MA-06,0.156564,dem,0.253964,NaN
4,1984,IL,0.018617,dem,-0.1822,0.200817,WV,WV-02,0.119416,dem,0.301616,-0.0077


In [32]:
# advantage_df.to_csv('data/advantages.csv', index=False)

---

### Interactive plot

In [33]:
def code_shape(inst):
    if inst == 'senate':
        return 'triangle'
    elif inst == 'house':
        return 'square'
    elif inst == 'ec':
        return 'circle'
    else:
        return None

In [34]:
df_final = pd.read_csv('data/advantages.csv')
# reshape
df_final = df_final[['year', 'senate_advantage', 'house_advantage', 'ec_advantage']].melt(id_vars='year', 
                                                                                           var_name='institution',
                                                                                           value_name='bias')
df_final.institution = df_final.institution.apply(lambda x: x.split('_')[0])
df_final['marker'] = df_final.institution.apply(code_shape)
df_final = df_final.round(4)
df_final['bias_display'] = df_final.bias.apply(lambda x: str(round((x*100), 1)) + '%')
df_final['institution_display'] = df_final.institution.replace({'ec': 'Electoral College', 'senate': 'U.S. Senate', 'house': 'U.S. House'})
df_final

,year,institution,bias,marker,bias_display,institution_display
0,1976,senate,0.0864,triangle,8.6%,U.S. Senate
1,1978,senate,-0.0259,triangle,-2.6%,U.S. Senate
2,1980,senate,0.0800,triangle,8.0%,U.S. Senate
3,1982,senate,0.2526,triangle,25.3%,U.S. Senate
4,1984,senate,0.2008,triangle,20.1%,U.S. Senate
...,...,...,...,...,...,...
64,2012,ec,0.0150,circle,1.5%,Electoral College
65,2014,ec,NaN,circle,nan%,Electoral College
66,2016,ec,-0.0281,circle,-2.8%,Electoral College
67,2018,ec,NaN,circle,nan%,Electoral College


In [37]:
color_mapper = LinearColorMapper(palette=tuple(reversed(bokeh.palettes.RdBu11)), 
                                 low=-max(-df_final.bias.min(), df_final.bias.max()),
                                 high=max(-df_final.bias.min(), df_final.bias.max()))

plot2 = figure(title='Electoral Structual Biases Over Time',
             x_axis_label='Year',
             y_axis_label='Dem. advantage (relative to national Dem. margin)',
             width=800,
             height=500)
r1 = plot2.scatter('year', 'bias', marker='marker', source=df_final[df_final.institution == 'house'],
               line_color="black", 
               color={'field': 'bias', 'transform': color_mapper},  
               fill_alpha=0.8, 
               size=10,
               legend_label='U.S. House')
r2 = plot2.scatter('year', 'bias', marker='marker', source=df_final[df_final.institution == 'senate'],
               line_color="black", 
               color={'field': 'bias', 'transform': color_mapper},  
               fill_alpha=0.8, 
               size=10,
               legend_label='U.S. Senate')
r3 = plot2.scatter('year', 'bias', marker='marker', source=df_final[df_final.institution == 'ec'],
               line_color="black", 
               color={'field': 'bias', 'transform': color_mapper},  
               fill_alpha=0.8, 
               size=17,
               legend_label='Electoral College')
plot2.add_tools(HoverTool(
                tooltips=[
                    ('Election Year', '@year'),
                    ('Institution', '@institution_display'),
                    ('Bias', '@bias_display')
                ]
            ))
plot2.legend.click_policy="hide"

show(plot2)

In [38]:
plot_list = [plot1, plot2]
bokeh.plotting.output_file('index.html')
show(bokeh.layouts.gridplot(plot_list, ncols=1))